...

In [22]:
from torchvision import datasets, transforms
import torch
from torch import nn
from torch.utils.data import DataLoader

from sklearn.manifold import TSNE
import pandas as pd
import seaborn as sns

In [23]:
batch_size = 1
device = "cuda"
print(torch.cuda.is_available())

True


In [24]:

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        ) 

    def forward(self, x):
        x = self.flatten(x)
        embedding = self.linear_relu_stack[:-1](x)  
        logits = self.linear_relu_stack(x)
        return embedding, logits

In [19]:
# [(num+15, train_data[i][1]) for num, i in enumerate(range(15, 40))]

In [25]:
from pprint import pprint
estado_fase_0 = torch.load('Fase_0.pth')
estado_fase_1 = torch.load('Fase_1.pth')

estado_fase_1["linear_relu_stack.4.weight"][5:10, :] = estado_fase_0["linear_relu_stack.4.weight"][5:10, :]
estado_fase_1["linear_relu_stack.4.bias"][5:10] = estado_fase_0["linear_relu_stack.4.bias"][5:10]

In [26]:
print("Weights Norm:")
wn = [(indice ,torch.norm(row) ) for indice ,row in enumerate(estado_fase_1["linear_relu_stack.4.weight"])]
pprint(wn)
print("Bias:")
bias = [(indice ,row ) for indice ,row in enumerate(estado_fase_1["linear_relu_stack.4.bias"])]
pprint(bias)

Weights Norm:
[(0, tensor(1.0947, device='cuda:0')),
 (1, tensor(1.2046, device='cuda:0')),
 (2, tensor(1.0862, device='cuda:0')),
 (3, tensor(1.0223, device='cuda:0')),
 (4, tensor(1.0812, device='cuda:0')),
 (5, tensor(0.6414, device='cuda:0')),
 (6, tensor(0.6275, device='cuda:0')),
 (7, tensor(0.6408, device='cuda:0')),
 (8, tensor(0.6609, device='cuda:0')),
 (9, tensor(0.6577, device='cuda:0'))]
Bias:
[(0, tensor(-0.0617, device='cuda:0')),
 (1, tensor(0.2236, device='cuda:0')),
 (2, tensor(-0.0377, device='cuda:0')),
 (3, tensor(-0.0188, device='cuda:0')),
 (4, tensor(0.0829, device='cuda:0')),
 (5, tensor(-0.1506, device='cuda:0')),
 (6, tensor(-0.1318, device='cuda:0')),
 (7, tensor(-0.1273, device='cuda:0')),
 (8, tensor(-0.1563, device='cuda:0')),
 (9, tensor(-0.1289, device='cuda:0'))]


In [27]:
model = NeuralNetwork().to(device)
model.load_state_dict(estado_fase_1)

<All keys matched successfully>

In [28]:

from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms

eval_data = torchvision.datasets.MNIST(
    root="../data",
    train=False,
    download=True,
    transform=transforms.ToTensor()
)
# Conjunto de evaluación
eval_indices_to_7 = [i for i in range(len(eval_data)) if eval_data.targets[i] == 7]

eval_to_7 = torch.utils.data.Subset(eval_data, eval_indices_to_7)
eval_to_7_dataloader = DataLoader(eval_to_7, batch_size, shuffle=True)

In [33]:
def extractor(dataloader, model):
    model.eval()
    embeddings = list()
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            features, logist_ = model(X)
            embeddings.append( (features.cpu(), logist_.cpu(),y.cpu()) )
            
    return embeddings

In [34]:
embeddings =  extractor(eval_to_7_dataloader, model)

In [39]:
embeddings[0][1]

tensor([[-2.8558, -3.0570, -4.6165, -1.7875, -0.8489, -1.1312, -1.7619,  0.3960,
         -0.9822, -1.2929]])